In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import mm_functions as mmf
import params as pr

In [ ]:
# Data storage location
prefix_dir = '/scratch2/raphaelr/processed_data/'

In [ ]:
# Compute and save data
def preprocess_data(var_name,cluster_number,n_storm,n_proc):
       
    # Open track datasets
    df_tracks, df_clust = mmf.open_tracks()
    # Select storms to be composited
    df_select = mmf.get_storms(df_tracks,df_clust,cluster_number,n_storm)
    # Make array of datetimes
    var_time = mmf.make_var_time(df_select)

    # variable-specific adjustments
    if (var_name == 'duaod550')or(var_name == 'pm10'): anom_flag = 1
    else: anom_flag = 0        
    ds_mask = mmf.make_mask(var_name,var_time)

    # Compute
    comp, dens = mmf.composite_var(var_name,df_select,var_time,ds_mask,anom_flag)
    if pr.res_flag=='plot':
        fn = prefix_dir+var_name+'/'+var_name+'_hr_bool_c'+str(cluster_number)+'.npz'
        # Save
        np.savez(fn, comp=comp, dens=dens)
    elif pr.res_flag=='sig':
        fn = prefix_dir+var_name+'/'+var_name+'_bool_c'+str(cluster_number)+'.npz'
        # Save
        np.savez(fn, comp=comp, dens=dens)
        # MC compute (save is internal)
        mmf.mc_composite_var(var_name,df_select,ds_mask,anom_flag,n_proc,prefix_dir,cluster_number)
    
    print(datetime.datetime.now())
    return

In [ ]:
# Select storms based on the cluster 
cluster_vec = np.arange(1,9+1) # Select cluster number from 1 to 12. Select 0 for all clusters.
n_storm = 0 # Enter 0 to composite all storms.
n_proc = 15 # Number of processors for the Monte-Carlo analysis

# Define the list of variables to be pre-processed
var_list = ['WCB','cold_front','DI']
# var_list = ['t2m','pm10']
# var_list = ['precip','wind_mag','swh','precip_mask'] # 4 variables in 16 hours on 15 cores

for cluster_number in cluster_vec:
    for var_name in var_list:
        preprocess_data(var_name,cluster_number,n_storm,n_proc)